In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("spark://data-eng-vm.asia-south2-a.c.data-engineering-rj.internal:7077") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/03/11 07:14:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
spark

In [3]:
df_green = spark.read.parquet('data/pq/green/*/*')

23/03/11 07:15:49 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/11 07:16:04 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/11 07:16:19 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/11 07:16:34 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/11 07:16:49 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
23/03/11 07:17:04 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure th

In [4]:
df_green = df_green \
    .withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

In [5]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [6]:
df_yellow = df_yellow \
    .withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime') \
    .withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

In [7]:
common_colums = []

yellow_columns = set(df_yellow.columns)

for col in df_green.columns:
    if col in yellow_columns:
        common_colums.append(col)

In [8]:
common_colums

['VendorID',
 'pickup_datetime',
 'dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'congestion_surcharge']

In [9]:
from pyspark.sql import functions as F

In [10]:
df_green_sel = df_green \
    .select(common_colums) \
    .withColumn('service_type', F.lit('green'))

In [11]:
df_green_sel.head()

Row(VendorID=32898, pickup_datetime=None, dropoff_datetime=datetime.datetime(2020, 1, 4, 14, 25, 5), store_and_fwd_flag='2020-01-04 14:39:39', RatecodeID=None, PULocationID=None, DOLocationID=42, passenger_count=50, trip_distance=1.0, fare_amount=5.06, extra=17.5, mta_tax=0.0, tip_amount=0.5, tolls_amount=5.0, improvement_surcharge=None, total_amount=0.3, payment_type=None, congestion_surcharge=1.0, service_type='green')

In [12]:
df_yellow_sel = df_yellow \
    .select(common_colums) \
    .withColumn('service_type', F.lit('yellow'))

In [13]:
df_trips_data = df_green_sel.unionAll(df_yellow_sel)

In [14]:
df_trips_data.groupBy('service_type').count().show()

+------------+--------+
|service_type|   count|
+------------+--------+
|       green| 4038568|
|      yellow|39649199|
+------------+--------+



In [23]:
df_trips_data.show(5)

+--------+---------------+-------------------+-------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|VendorID|pickup_datetime|   dropoff_datetime| store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|payment_type|congestion_surcharge|service_type|
+--------+---------------+-------------------+-------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------------------+------------+------------+--------------------+------------+
|   32898|           null|2020-01-04 14:25:05|2020-01-04 14:39:39|      null|        null|          42|             50|          1.0|       5.06| 17.5|    0.0|       0.5|         5.0|                 nu

In [15]:
df_trips_data.registerTempTable('trips_data')

/home/rohit/spark/spark-3.2.3-bin-hadoop3.2/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [16]:
spark.sql("""
SELECT
    service_type,
    count(1)
FROM
    trips_data
GROUP BY 
    service_type
""").show()

+------------+--------+
|service_type|count(1)|
+------------+--------+
|       green| 4038568|
|      yellow|39649199|
+------------+--------+



In [17]:
df_result = spark.sql("""
SELECT 
    -- Reveneue grouping 
    PULocationID AS revenue_zone,
    date_trunc('month', pickup_datetime) AS revenue_month, 
    service_type, 

    -- Revenue calculation 
    SUM(fare_amount) AS revenue_monthly_fare,
    SUM(extra) AS revenue_monthly_extra,
    SUM(mta_tax) AS revenue_monthly_mta_tax,
    SUM(tip_amount) AS revenue_monthly_tip_amount,
    SUM(tolls_amount) AS revenue_monthly_tolls_amount,
    SUM(improvement_surcharge) AS revenue_monthly_improvement_surcharge,
    SUM(total_amount) AS revenue_monthly_total_amount,
    SUM(congestion_surcharge) AS revenue_monthly_congestion_surcharge,

    -- Additional calculations
    AVG(passenger_count) AS avg_montly_passenger_count,
    AVG(trip_distance) AS avg_montly_trip_distance
FROM
    trips_data
GROUP BY
    1, 2, 3
""")

In [18]:
df_result.show(5)

+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|revenue_zone|      revenue_month|service_type|revenue_monthly_fare|revenue_monthly_extra|revenue_monthly_mta_tax|revenue_monthly_tip_amount|revenue_monthly_tolls_amount|revenue_monthly_improvement_surcharge|revenue_monthly_total_amount|revenue_monthly_congestion_surcharge|avg_montly_passenger_count|avg_montly_trip_distance|
+------------+-------------------+------------+--------------------+---------------------+-----------------------+--------------------------+----------------------------+-------------------------------------+----------------------------+------------------------------------+--------------------------+------------------------+
|         250|2020-

23/03/11 07:29:36 ERROR TaskSchedulerImpl: Lost executor 0 on 10.190.0.2: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
23/03/11 07:29:49 WARN StandaloneAppClient$ClientEndpoint: Connection to data-eng-vm.asia-south2-a.c.data-engineering-rj.internal:7077 failed; waiting for master to reconnect...
23/03/11 07:29:49 WARN StandaloneSchedulerBackend: Disconnected from Spark cluster! Waiting for reconnection...
23/03/11 07:29:49 WARN StandaloneAppClient$ClientEndpoint: Connection to data-eng-vm.asia-south2-a.c.data-engineering-rj.internal:7077 failed; waiting for master to reconnect...


In [35]:
df_result.coalesce(1).write.parquet('data/report/revenue/', mode='overwrite')